In [30]:
!jupyter nbconvert --to script DistributedDataParallel_script.ipynb
!python DistributedDataParallel_script.py --issingle
!rm -rf MNIST_data

[NbConvertApp] Converting notebook DistributedDataParallel_script.ipynb to script
[NbConvertApp] Writing 7842 bytes to DistributedDataParallel_script.py
Dataset MNIST
    Number of datapoints: 60000
    Root location: ./MNIST_data
    Split: Train
(784,) 5
args =  Namespace(batch=16, epochs=1, issingle=True, lr=0.001, rank=None, size=None)
Single GPU training
using device cuda
100%|██████████████████████████████████████| 3750/3750 [00:12<00:00, 296.45it/s]
loss at epoch 0 is 0.007238924503326416
shape = torch.Size([128, 784]), mean of values = -0.002532262122258544
shape = torch.Size([128]), mean of values = 0.021755723282694817
shape = torch.Size([64, 128]), mean of values = 0.009082158096134663
shape = torch.Size([64]), mean of values = 0.030485646799206734
shape = torch.Size([10, 64]), mean of values = -0.027527540922164917
shape = torch.Size([10]), mean of values = 0.003641219809651375


## Initialize Azure ML

In [31]:
from azureml.core import Workspace, Experiment, Run
ws = Workspace(subscription_id='<Enter>',
              resource_group='<>',
              workspace_name='SubstrateIntelligenceNLR-WS2')

experiment = Experiment(ws, 'MNIST-singlegpu')

from azureml.core import ComputeTarget
compute_target = ComputeTarget(ws, 'nc24sv3')

### Single GPU

In [48]:
from azureml.train.dnn import PyTorch
estimator = PyTorch(source_directory = '.',
                    pip_packages = ['tqdm'],
                    compute_target = compute_target,
                    entry_script = 'DistributedDataParallel_script.py',
                    script_params = {
                       '--issingle':""
                    },
                    use_gpu=True,
                   )

run = experiment.submit(estimator)

WARNING - framework_version is not specified, defaulting to version 1.1.


In [49]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

### Distributed data parallel AML job - Not working
2 nodes 4 GPUS
Follow this
https://github.com/microsoft/AzureML-BERT/blob/master/pretrain/PyTorch/azureml_adapter.py

In [52]:
from azureml.train.dnn import PyTorch
from azureml.core.runconfig import MpiConfiguration
mpi = MpiConfiguration()
mpi.process_count_per_node = 4

estimator_distrib = PyTorch(source_directory = '.',
                    pip_packages = ['tqdm'],
                   compute_target = compute_target,
                   entry_script = 'DistributedDataParallel_script.py',
                   script_params = {},
                   use_gpu=True,
                   distributed_training = mpi,
                    node_count = 2
                   )

run_distrib = experiment.submit(estimator_distrib)

WARNING - framework_version is not specified, defaulting to version 1.1.


In [53]:
RunDetails(run_distrib).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…